In [726]:
from typing import Any, List, Generator, Union, Optional
import re




In [727]:

class Expr:
    """Represents logical expressions using operators and arguments."""

    def __init__(self, op: Union[str, int], *args: Any) -> None:
        self.op = op
        self.args = list(map(Expr.create_expression, args)) ## Coerce args to Exprs

    @staticmethod
    def create_expression(s: Union[str, int]) -> "Expr":
        if isinstance(s, Expr):
            return s
        if s.isnumeric():
            return Expr(s)
        s = s.replace("==>", ">>").replace("<==", "<<")
        s = s.replace("<=>", "%").replace("=/=", "^")
        s = re.sub(r'([a-zA-Z0-9_.]+)', r'Expr("\1")', s)
        return eval(s, {"Expr": Expr})
    
    def __call__(self, *args):
        """Self must be a symbol with no args, such as Expr('F').  Create a new
        Expr with 'F' as op and the args as arguments."""
        assert Logic().is_symbol(self.op) and not self.args
        return Expr(self.op, *args)

    def __repr__(self):
        "Show something like 'P' or 'P(x, y)', or '~P' or '(P | Q | R)'"
        if not self.args:         # Constant or proposition with arity 0
            return str(self.op)
        elif Logic().is_symbol(self.op):  # Functional or propositional operator
            return '%s(%s)' % (self.op, ', '.join(map(repr, self.args)))
        elif len(self.args) == 1: # Prefix operator
            return self.op + repr(self.args[0])
        else:                     # Infix operator
            return '(%s)' % (' '+self.op+' ').join(map(repr, self.args))

    def __eq__(self, other):
        """x and y are equal iff their ops and args are equal."""
        return (other is self) or (isinstance(other, Expr)
            and self.op == other.op and self.args == other.args)

    def __ne__(self, other):
        return not self.__eq__(other)

    def __hash__(self):
        "Need a hash method so Exprs can live in dicts."
        return hash(self.op) ^ hash(tuple(self.args))
    
    
    def __lt__(self, other):     return Expr('<',  self, other)
    def __le__(self, other):     return Expr('<=', self, other)
    def __ge__(self, other):     return Expr('>=', self, other)
    def __gt__(self, other):     return Expr('>',  self, other)
    def __add__(self, other):    return Expr('+',  self, other)
    def __sub__(self, other):    return Expr('-',  self, other)
    def __and__(self, other):    return Expr('&',  self, other)
    def __div__(self, other):    return Expr('/',  self, other)
    def __truediv__(self, other):return Expr('/',  self, other)
    def __invert__(self):        return Expr('~',  self)
    def __lshift__(self, other): return Expr('<<', self, other)
    def __rshift__(self, other): return Expr('>>', self, other)
    def __mul__(self, other):    return Expr('*',  self, other)
    def __neg__(self):           return Expr('-',  self)
    def __or__(self, other):     return Expr('|',  self, other)
    def __pow__(self, other):    return Expr('**', self, other)
    def __xor__(self, other):    return Expr('^',  self, other)
    def __mod__(self, other):    return Expr('<=>',  self, other)



In [728]:

class Logic:
    """A class for logical operations and conversions."""

    TRUE, FALSE, ZERO, ONE, TWO = map(Expr, ["TRUE", "FALSE", 0, 1, 2])
    A, B, C, D, E, F, G, P, Q, x, y, z = map(Expr, "ABCDEFGPQxyz")
    _op_identity = {"&": TRUE, "|": FALSE, "+": ZERO, "*": ONE}
    


    def is_symbol(self, s: str) -> bool:
        """Returns True if the string is a symbol."""
        return isinstance(s, str) and s[:1].isalpha()

    def is_var_symbol(self, s: str) -> bool:
        """Returns True if the string is a variable symbol."""
        return self.is_symbol(s) and s[0].islower()

    def is_prop_symbol(self, s: str) -> bool:
        """Returns True if the string is a proposition symbol."""
        return self.is_symbol(s) and s[0].isupper() and s != "TRUE" and s != "FALSE"

    def variables(self, s: "Expr") -> set:
        """Returns a set of variables in the expression."""
        result = set([])

        def walk(s):
            if self.is_var_symbol(s):
                result.add(s)
            else:
                for arg in s.args:
                    walk(arg)

        walk(s)
        return result

    def tt_entails(self, kb: "Expr", alpha: "Expr") -> bool:
        """Does kb entail the sentence alpha? Use truth tables. For propositional
        kb's and sentences."""
        assert not self.variables(alpha)
        return self.tt_check_all(kb, alpha, self.prop_symbols(kb & alpha), {})

    def tt_check_all(
        self, kb: "Expr", alpha: "Expr", symbols: list, model: dict
    ) -> bool:
        """Auxiliary routine to implement tt_entails."""
        if not symbols:
            if self.evaluate(kb, model):
                result = self.evaluate(alpha, model)
                assert result in (True, False)
                return result
            else:
                return True
        else:
            P, rest = symbols[0], symbols[1:]
            model_true = model.copy()
            model_true[P] = True
            model_false = model.copy()
            model_false[P] = False
            return (self.tt_check_all(kb, alpha, rest, model_true) and
                    self.tt_check_all(kb, alpha, rest, model_false))


    def prop_symbols(self, x: "Expr") -> set:
        """Return a set of all propositional symbols in x."""
        if not isinstance(x, Expr):
            return list()
        elif self.is_prop_symbol(x.op):
            return [x]
        else:
            return list(set(symbol for arg in x.args for symbol in self.prop_symbols(arg)))

    def evaluate(self, exp: "Expr", model: dict = {}) -> Optional[bool]:
        """Return True if the propositional logic expression is true in the model,
        and False if it is false. If the model does not specify the value for
        every proposition, this may return None to indicate 'not obvious';
        this may happen even when the expression is tautological."""
        op, args = exp.op, exp.args
        
        # If the expression is a constant, return its value
        if exp in (self.TRUE, self.FALSE):
            return exp == self.TRUE
        
        # If the expression is a proposition symbol, return its value
        if self.is_prop_symbol(op):
            return model.get(exp)
        
        # Evaluate complex expressions
        if op == "~": # negation
            p = self.evaluate(args[0], model)
            if p is None:
                return None
            else:
                return not p
        elif op == "|": # disjunction
            result = False
            for arg in args:
                p = self.evaluate(arg, model)
                if p is True:
                    return True
                if p is None:
                    result = None
            return result
        elif op == "&": # conjunction
            result = True
            for arg in args:
                p = self.evaluate(arg, model)
                if p is False:
                    return False
                if p is None:
                    result = None
            return result
        
        # At this point, the operator represents a binary operation
        # Split the expression into a list of clauses
        p, q = args
        
        if op == ">>": # implication
            return self.evaluate(~p | q, model)
        elif op == "<<": # reverse implication
            return self.evaluate(p | ~q, model)
        
        # If the operator is not an implication, evaluate the expression
        # Get the truth values of the two propositions
        pt = self.evaluate(p, model)
        qt = self.evaluate(q, model) 
        
        # If the truth values are not known, return None
        if not pt or not qt:
            return None
        
        # Evaluate the expression
        if op == "<=>": # biconditional
            return pt == qt
        elif op == "^": # exclusive or
            return pt != qt
        else:
            raise ValueError("illegal operator in logic expression" + str(exp))

    def to_cnf(self, s: "Expr") -> "Expr":
        """Converts a propositional logical sentence s to conjunctive normal form."""
        if isinstance(s, str):
            s = Expr.create_expression(s)
        s = self.eliminate_implications(s)
        s = self.move_not_inwards(s)
        return self.distribute_and_over_or(s)

    def eliminate_implications(self, s: "Expr") -> "Expr":
        """Change >>, <<, and <=> into &, |, and ~."""
        if not s.args or self.is_symbol(s.op):
            return s
        args = list(map(self.eliminate_implications, s.args))
        a, b = args[0], args[-1]
        if s.op == ">>":
            return b | ~a
        elif s.op == "<<":
            return a | ~b
        elif s.op == "<=>":
            return (a | ~b) & (b | ~a)
        elif s.op == "^":
            assert len(args) == 2
            return (a & ~b) | (~a & b)
        else:
            assert s.op in ("&", "|", "~")
            return Expr(s.op, *args)
    
    def move_not_inwards(self, s: "Expr") -> "Expr":
        """Rewrite sentence s by moving negation sign inward."""
        if s.op == "~":
            NOT = lambda b: self.move_not_inwards(~b)
            a = s.args[0]
            if a.op == "~":
                return self.move_not_inwards(a.args[0])
            if a.op == "&":
                return self.associate("|", map(NOT, a.args))
            if a.op == "|":
                return self.associate("&", map(NOT, a.args))
            return s
        elif self.is_symbol(s.op) or not s.args:
            return s
        else:
            return Expr(s.op, *map(self.move_not_inwards, s.args))

    def distribute_and_over_or(self, s: "Expr") -> "Expr":
        """Given a sentence s consisting of conjunctions and disjunctions of literals,
        return an equivalent sentence in CNF."""
        if s.op == "|":
            s = self.associate("|", s.args)
            if s.op != "|":
                return self.distribute_and_over_or(s)
            if len(s.args) == 0:
                return self.FALSE
            if len(s.args) == 1:
                return self.distribute_and_over_or(s.args[0])
            conj = next((d for d in s.args if d.op == "&"), None)
            if not conj:
                return s
            others = [a for a in s.args if a is not conj]
            rest = self.associate("|", others)
            return self.associate(
                "&", [self.distribute_and_over_or(c | rest) for c in conj.args]
            )
        elif s.op == "&":
            return self.associate("&", map(self.distribute_and_over_or, s.args))
        else:
            return s

    def associate(self, op: str, args: List["Expr"]) -> "Expr":
        """Given an associative op, return an expression with the same meaning as
        Expr(op, *args), but flattened."""
        args = self.dissociate(op, args)
        if len(args) == 0:
            return self._op_identity[op]
        elif len(args) == 1:
            return args[0]
        else:
            return Expr(op, *args)

    def dissociate(self, op: str, args: List["Expr"]) -> List["Expr"]:
        """Given an associative op, return a flattened list result such that
        Expr(op, *result) means the same as Expr(op, *args)."""
        result = []

        def collect(subargs):
            for arg in subargs:
                if arg.op == op:
                    collect(arg.args)
                else:
                    result.append(arg)

        collect(args)
        return result

    def conjuncts(self, s: "Expr") -> List["Expr"]:
        """Return a list of the conjuncts in the sentence s."""
        return self.dissociate("&", [s])
    
    def disjuncts(self, s: "Expr") -> List["Expr"]:
        """Return a list of the disjuncts in the sentence s."""
        # print("DISJUNCTS", s)
        return self.dissociate("|", [s])
    
    

In [729]:


class KnowledgeBase:
    """A base class for Knowledge Base (KB) systems."""
    def __init__(self):
        self.clauses: List[Expr] = []

    def ask(self, query: Any) -> Union[dict, bool]:
        """Returns a substitution that makes the query true, or False if not found."""
        return Logic().tt_entails(Expr("&", *self.clauses), query)

    def tell(self, sentence: "Expr") -> None:
        """Adds clauses of a sentence to the KB."""
        cnf_sentence = Logic().to_cnf(sentence)
        conjuncts = Logic().conjuncts(cnf_sentence)
        self.clauses.extend(conjuncts)


    # def ask_generator(self, query: "Expr") -> Generator[dict, None, None]:
    #     """Yields an empty substitution if the KB implies the query."""
    #     # if Expr.tt_entails(Expr("&", *self.clauses), query):
    #     if Logic().tt_entails(Expr("&", *self.clauses), query):
    #         yield {}

    def retract(self, sentence: "Expr") -> None:
        """Removes clauses of a sentence from the KB."""
        
        for clause in Logic().conjuncts(Logic().to_cnf(sentence)):
            if clause in self.clauses:
                self.clauses.remove(clause)


In [730]:
import collections


class LogicAux:
    def normalize(self, clause: "Expr") -> "Expr":
        disjunctions = Logic().disjuncts(clause)
        mapped = map(str, disjunctions)
        return frozenset(mapped)
    
    def negate(self, literal: str) -> str:
        if literal[0] == "~":
            return literal[1:]
        else:
            return "~" + literal
        
        
    def add_parentheses(self, literal: str) -> str:
        if not literal.endswith(")"):
            return "(" + literal + ")"
        return literal
        
    
    # def resolution(self, KB: "KnowledgeBase", alpha: "Expr") -> bool:
    #     """Apply the resolution algorithm to determine if alpha can be inferred from KB."""
        
        
    #     tainted_clauses = set(self.normalize(clause) for clause in Logic().conjuncts(Logic().to_cnf(~alpha)))
    #     KB_clauses = [self.normalize(clause) for clause in KB.clauses]

        
    #     new = set()
    #     while True:
    #         # here we will resolve the clauses
    #         # we will first create a dictionary of clauses that contain each literal
    #         clausesWith = collections.defaultdict(list)
    #         for clause in list(tainted_clauses) + KB_clauses:
    #             for literal in clause:
    #                 # check if the literal ends with ")", if not surround it with parentheses
    #                 clausesWith[literal] = clause
                    
            
    #         # we will now create pairs of clauses that contain complementary literals
    #         pairs = []
    #         for clause0 in tainted_clauses:
    #             for literal in clause0:
    #                 negated_literal = self.negate(literal)
    #                 for clause1 in clausesWith[negated_literal]:
    #                     pairs.append((literal, clause0, clause1))
                        
    #                 # we need to search some keys that don't have parentheses
    #                 for key in clausesWith.keys():
    #                     if not key.endswith(")"):
    #                         # Add parentheses to the key
    #                         new_key = "(" + key + ")"
    #                         # Now check if the negated literal matches the key
    #                         if new_key == negated_literal:
    #                             for clause1 in clausesWith[key]:
    #                                 pairs.append((literal, clause0, clause1))
            
    #         # we will now resolve the pairs
    #         for literal, clause0, clause1 in pairs:
    #             result = self.resolve(clause0, clause1, literal)
    #             if result is not None:
    #                 if result == set():
    #                     return True
    #                 else:
    #                     new.add(frozenset(result))
            
    #         # check if the new clauses are already in the KB
    #         added = False
    #         for clause in new:
    #             if not any(old_clause.issubset(clause) for old_clause in list(tainted_clauses) + KB_clauses):
    #                 tainted_clauses.add(clause)
    #                 added = True
            
    #         # if no new clauses were added, return False because alpha cannot be inferred
    #         if not added:
    #             return False
            
    def resolve(self, clause0: "Expr", clause1: "Expr", literal: str) -> Optional[set]:
        """Resolve two clauses with respect to a literal."""
        
        # First, check if the clauses contain the literal and its negation
        # If they do, remove the literal from clause0 and its negation from clause1
        
        # print("IN RESOLVE", clause0, clause1, literal)
        clause0 = set(clause0)
        
        # Clause 1 is alwayys a string, add it to a set without splitting by character
        tmp_clause1 = set()
        tmp_clause1.add(clause1)
        clause1 = tmp_clause1
        
        # For every element in clause0 call the add_parentheses function
        for element in clause0:
            new_element = self.add_parentheses(element)
            clause0.remove(element)
            clause0.add(new_element)
            
            
        for element in clause1:
            new_element = self.add_parentheses(element)
            clause1.remove(element)
            clause1.add(new_element)
            
        
        try:
            clause0.remove(literal)
        except KeyError:
            pass
        
        try:
            clause0.remove(self.add_parentheses(literal))
        except KeyError:
            pass
        
        try: 
            clause1.remove(self.negate(literal))
        except KeyError:
            pass
        
        try:
            clause1.remove(self.add_parentheses(self.negate(literal)))
        except KeyError:
            pass
        
        
        # Check if the clauses are resolvable
        if any(self.negate(other) in clause1 for other in clause0):
            return None
        
        # Return the union of the two clauses
        return clause0.union(clause1)
    
    
    



In [731]:
import random

class Environment:
    def __init__(self, grid_size: int = 4, player_position: tuple = (0, 0)):
        # the map is always a square
        self.grid_size = grid_size
        self.player_position = player_position
        # self.generate_map()
        self.map = [
            ['o', 'o', 'o', 'P'],
            ['W', 'G', 'P', 'o'],
            ['o', 'o', 'o', 'o'],
            ['o', 'o', 'P', 'o']
        ]
        
    
    def get_random_position(self, start: int = 0, end: int = None):
        return (random.randint(start, end - 1), random.randint(start, end - 1))
        
    def generate_map(self):
        self.map = [['o' for _ in range(self.grid_size)] for _ in range(self.grid_size)]
        
        # Place the wumpus
        wumpus_position = self.get_random_position(0, self.grid_size)
        while wumpus_position == self.player_position:
            wumpus_position = self.get_random_position(0, self.grid_size)
        self.map[wumpus_position[0]][wumpus_position[1]] = 'W'
        
        # Place the gold
        gold_position = self.get_random_position(0, self.grid_size)
        while gold_position == self.player_position or gold_position == wumpus_position:
            gold_position = self.get_random_position(0, self.grid_size)
        self.map[gold_position[0]][gold_position[1]] = 'G'
        
        # Place the pits
        for y in range(self.grid_size):
            for x in range(self.grid_size):
                if (x, y) == self.player_position or (x, y) == wumpus_position or (x, y) == gold_position:
                    continue
                if random.random() < 0.2:
                    self.map[y][x] = 'P'
    
    def get_map(self):
        return self.map
    
    def print_map(self, player_position: tuple = None):
        for x in range(self.grid_size):
            for y in range(self.grid_size):
                if player_position is not None and (x, y) == player_position:
                    print('♥', end=' ')
                else:
                    print(self.map[x][y], end=' ')
            print()
    
    def get_nearby_cells(self, position):
        # Only return cells that are within the bounds of the map
        x, y = position
        nearby_cells = []
        for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < len(self.map[0]) and 0 <= new_y < len(self.map):
                nearby_cells.append((new_x, new_y))
        return nearby_cells
    
    def get_percept(self, position):
        percept = [None, None, None, None, None]
        nearby_cells = self.get_nearby_cells(position)
        
        # Check if the player perceives a bump
        if not self.is_valid_position(position):
            percept[3] = 'Bump'
        
        # Only check the rest of the percepts if the player did not perceive a bump
        if percept[3] is None:
            # Check for pits, wumpus, and gold in nearby cells
            for cell in nearby_cells:
                if self.is_pit(cell):
                    percept[1] = 'Breeze'
                if self.is_wumpus(cell):
                    percept[0] = 'Stench'
            
            # Check for gold in the current cell
            if self.is_gold(position):
                percept[2] = 'Glitter'
                
            
            
            
            # Check if the wumpus is alive
            if not self.is_wumpus_alive():
                percept[4] = 'Scream'

        return percept
    
    def is_valid_position(self, position):
        x, y = position
        return 0 <= x < len(self.map) and 0 <= y < len(self.map[0])
    
    def is_pit(self, position):
        x, y = position
        return self.map[x][y] == 'P'
    
    def is_wumpus(self, position):
        x, y = position
        return self.map[x][y] == 'W'
    
    def is_gold(self, position):
        x, y = position
        return self.map[x][y] == 'G'
    
    def is_wumpus_alive(self):
        return any('W' in row for row in self.map)
    
    def remove_wumpus(self):
        for x in range(len(self.map)):
            for y in range(len(self.map[0])):
                if self.map[x][y] == 'W':
                    self.map[x][y] = 'o'
                    return
                
    def remove_gold(self):
        for x in range(len(self.map)):
            for y in range(len(self.map[0])):
                if self.map[x][y] == 'G':
                    self.map[x][y] = 'o'
                    return

In [732]:

class LogicAgent:
    def __init__(self):
        self.KB = KnowledgeBase()
        self.Logic = Logic()
        self.LogicAux = LogicAux()
        self.DELTAS = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        
    def safe_neighbors(self, position):
        neighbours = self.get_neighbors(position)        
        safe_neighbors = set()
        
        for neighbor in neighbours:
            x, y = neighbor
            
            if x < 0 or y < 0:
                continue
            
            # Current Neighbor
            loc = f"{x}_{y}"
            
            # Add current location spot to the safe_spots
            is_loc = self.KB.ask(Expr.create_expression(f"L{loc}"))
            if is_loc:
                safe_neighbors.add(neighbor)
            
            # print(f"Safe Neighbors: {safe_neighbors} after checking for location")
            isnt_pit = self.KB.ask(Expr.create_expression(f"~P{loc}"))
            isnt_wumpus = self.KB.ask(Expr.create_expression(f"~W{loc}"))
            if isnt_pit and isnt_wumpus:
                safe_neighbors.add(neighbor)
            
            # print("Current KB: ", self.KB.clauses)
            print(f"Current neighbor: {neighbor} - {loc} - isnt_pit: {isnt_pit} - isnt_wumpus: {isnt_wumpus} is_loc: {is_loc}")
        return safe_neighbors                
        
    def not_unsafe_neighbors(self, position: tuple):
        neighbours = self.get_neighbors(position)
        not_unsafe_neighbors = set()
        
        for neighbor in neighbours:
            x, y = neighbor
            
            if x < 0 or y < 0:
                continue
            
            # Current Neighbor
            loc = f"{x}_{y}"
            
            # if not self.LogicAux.resolution(self.KB, Expr.create_expression(f"L{loc}")):
            is_loc = self.KB.ask(Expr.create_expression(f"L{loc}"))
            if not is_loc:
                not_unsafe_neighbors.add(neighbor)
                
            
            # # Not a pit or not a wumpus at current location
            # has_wumpus = self.LogicAux.resolution(self.KB, Expr.create_expression(f"W{loc}"))
            is_wumpus = self.KB.ask(Expr.create_expression(f"W{loc}"))
            is_pit = self.KB.ask(Expr.create_expression(f"P{loc}"))
            if not is_wumpus and not is_pit:
                if neighbor in not_unsafe_neighbors:
                    not_unsafe_neighbors.remove(neighbor)

        return not_unsafe_neighbors
        
        
    def get_neighbors(self, position):
        return [(position[0] + dx, position[1] + dy) for dx, dy in self.DELTAS if 0 <= position[0] + dx and 0 <= position[1] + dy]
    
    def unvisited(self, position):
        """
        Use logic to determine the set of locations I have visited.
        """
        # for testing, also ask for current location
        result = set()
        neighbours = self.get_neighbors(position)
        for x, y in neighbours:
            if x < 0 or y < 0:
                continue
            
            is_loc = self.KB.ask(Expr.create_expression(f"L{x}_{y}"))
            # print(f"KB is {self.KB.clauses}")
            # print(f"Checking if {x}_{y} is a location: {is_loc}, but we need not is_loc, so: {not is_loc}")
            if not is_loc:
                result.add((x, y))
        
        return result
    
    def get_next_cell(self, position):
        print(f"Current Position: {position}")  
        unvisited_cells = self.unvisited(position)
        print(f"Unvisited Cells: {unvisited_cells}")
        
        safe_cells = self.safe_neighbors(position)
        print(f"Safe Cells: {safe_cells}")
        
        safe_cells = safe_cells.intersection(unvisited_cells)
        
        print(f"Safe Unvisited Cells: {safe_cells}")
        
        if safe_cells:
            next_cell = min(safe_cells)
        else:
            print(f"Not unsafe Cells: {self.not_unsafe_neighbors(position)}")
            not_unsafe_cells = self.not_unsafe_neighbors(position).intersection(unvisited_cells)
            if not_unsafe_cells:
                next_cell = min(not_unsafe_cells)
            else:
                raise Exception("Nowhere left to go")

        return next_cell

class WumpusAgent(LogicAgent):
    def __init__(self, initial_position=(0, 0)):
        # Initialize the agent
        super().__init__()
        
        # Constants
        self.ORIENTATIONS = ['NORTH', 'EAST', 'SOUTH', 'WEST']
        self.MOVEMENTS = {'NORTH': (0, 1), 'EAST': (1, 0), 'SOUTH': (0, -1), 'WEST': (-1, 0)}
        
        # Initialize agent state
        self.initial_position = initial_position
        self.position = initial_position
        self.orientation = 'EAST'
        self.has_gold = False
        self.has_arrow = True
        self.t = 0 # Time step
        self.is_alive = True
        self.score = 0
        self.wumpus_alive = True


    def make_action_sentence(self, action):
        return f"Executed({action}| {self.position}| {self.orientation}| {self.t})"
    
    def make_clause(self, position, symbol):
        return f"{symbol}{position[0]}_{position[1]}"
    
    def make_neighbor_clause(self, position, symbol, consecuence_symbol, negation=False):
        conjunction = "&" if negation else "|"
        x, y = position
        
        clause = f"{symbol}{x}_{y} <=> ("
        neighbors = self.get_neighbors(position)
        for n in neighbors:
            clause += f" {consecuence_symbol}{n[0]}_{n[1]} {conjunction}"
        clause = clause[:-1] + ")"
        return clause
 
    def make_percept_knowledge(self, percept, t):
        # Create sentences based on the percept
        percept_sentences = []
        
        
        # Scream
        if 'Scream' in percept:
            self.wumpus_alive = False
            
        # New perceptions
        # The obvious ones
        # 1. Current location
        percept_sentences.append(self.make_clause(self.position, 'L'))
        
        # 2. No pit at current location
        percept_sentences.append(self.make_clause(self.position, '~P'))
        
        # 3. No wumpus at current location
        percept_sentences.append(self.make_clause(self.position, '~W'))
        
        # Breeze
        if 'Breeze' in percept:
            breeze = self.make_clause(self.position, 'B')
            neighbor_clause = self.make_neighbor_clause(self.position, 'B', 'P')
        else:
            breeze = self.make_clause(self.position, '~B')
            neighbor_clause = self.make_neighbor_clause(self.position, '~B', '~P', negation=True)
        percept_sentences.append(breeze)
        percept_sentences.append(neighbor_clause)
        
        # Stench
        if 'Stench' in percept:
            stench = self.make_clause(self.position, 'S')
            neighbor_clause = self.make_neighbor_clause(self.position, 'S', 'W')
        else:
            stench = self.make_clause(self.position, '~S')
            neighbor_clause = self.make_neighbor_clause(self.position, '~S', '~W', negation=True)
        percept_sentences.append(stench)
        percept_sentences.append(neighbor_clause)
        
        # Glitter
        if 'Glitter' in percept:
            percept_sentences.append(self.make_clause(self.position, 'G'))
        
        return percept_sentences
    
    def go_to(self, goal):
        self.position = goal
    
    def act(self, percept):
        # Update time step
        self.t += 1
        
        # Get the next action
        knowledge = self.make_percept_knowledge(percept, self.t)
        
        # If we feel a bump, return to the previous position
        if 'Bump' in percept:
            self.KB.tell(f"L{self.position[0]}_{self.position[1]}")
            self.KB.tell(f"N{self.position[0]}_{self.position[1]}")
            self.position = (self.position[0] - self.MOVEMENTS[self.orientation][0], self.position[1] - self.MOVEMENTS[self.orientation][1])
            return self.make_action_sentence('Return')
        
        # Check if there is a Glitter 
        if 'Glitter' in percept and not self.has_gold:
            self.has_gold = True
            self.KB.tell(Expr.create_expression(f"G{self.position[0]}_{self.position[1]}"))
            return self.make_action_sentence('Grab')
        
        # If we have gold we need to go back to the start
        if self.has_gold and self.position == self.initial_position:
            return self.make_action_sentence('Climb')
        elif self.has_gold:
            self.go_to(self.initial_position)
            return self.make_action_sentence('Forward')
        
        # Update knowledge base
        for sentence in knowledge:
            self.KB.tell(sentence)
        
        # Ask the knowledge base for the next action
        next_cell = self.get_next_cell(self.position)
        
        # Change the orientation and move to the next cell
        dx, dy = next_cell[0] - self.position[0], next_cell[1] - self.position[1]
        
        # Locate the orientation
        new_orientation = [k for k, v in self.MOVEMENTS.items() if v == (dx, dy)][0]
        self.orientation = new_orientation
        
        # Move to the next cell
        self.position = next_cell
        
        # Return the action
        return self.make_action_sentence('Forward')

In [733]:
# Utils

def get_position_string(action):
    actions = action.split("|")
    position_string = actions[1].strip() # (x, y)
    positions = position_string.split(",") # ['(x', 'y)']
    positions = list(map(lambda x: x.strip(), positions)) # ['(x', 'y)']
    x, y = int(positions[0][1:]), int(positions[1][:-1])
    return x, y

def get_last_action(action):
    actions = action.split("|")
    actions = actions[0].split("(")
    return actions[1].strip()

In [734]:
# Initialize the world and the agent
world = Environment()
start_player_position = (len(world.map) - 1, 0)
agent = WumpusAgent(start_player_position)

# Lets play
for _ in range(50):
    # Perceive
    percept = world.get_percept(agent.position)
    print(percept)
    print(agent.KB.clauses)
    world.print_map(agent.position)
    
    # Act
    action = agent.act(percept)
    print(action)
    
    # If there is a "position" in the action, we can extract it as its going to be "... position (x, y)..."
    x, y = get_position_string(action)
    last_action = get_last_action(action)
    print(last_action)
    
    if last_action == "Climb":
        print("Game Over, you have climbed out of the cave")
        break
    elif last_action == "Grab":
        print("Gold has been grabbed")
        world.remove_gold()
    
    # Check if player is located at a pit or wumpus
    if world.is_valid_position((x, y)) and (world.is_pit((x, y)) or world.is_wumpus((x, y))):
        if world.is_pit((x, y)):
            print("Fell into a pit")
        else:
            print("Wumpus killed you")
        agent.is_alive = False
        print("Game over")
        break
    
    print("-----------------")
    # self.map = 


[None, None, None, None, None]
[]
o o o P 
W G P o 
o o o o 
♥ o P o 
Current Position: (3, 0)
Unvisited Cells: {(3, 1), (4, 0), (2, 0)}
Current neighbor: (3, 1) - 3_1 - isnt_pit: True - isnt_wumpus: True is_loc: False
Current neighbor: (4, 0) - 4_0 - isnt_pit: True - isnt_wumpus: True is_loc: False
Current neighbor: (2, 0) - 2_0 - isnt_pit: True - isnt_wumpus: True is_loc: False
Safe Cells: {(3, 1), (4, 0), (2, 0)}
Safe Unvisited Cells: {(3, 1), (4, 0), (2, 0)}
Executed(Forward| (2, 0)| WEST| 1)
Forward
-----------------
['Stench', None, None, None, None]
[L3_0, ~P3_0, ~W3_0, ~B3_0, (~B3_0 | P3_1 | P4_0 | P2_0), (~P3_1 | B3_0), (~P4_0 | B3_0), (~P2_0 | B3_0), ~S3_0, (~S3_0 | W3_1 | W4_0 | W2_0), (~W3_1 | S3_0), (~W4_0 | S3_0), (~W2_0 | S3_0)]
o o o P 
W G P o 
♥ o o o 
o o P o 
Current Position: (2, 0)
Unvisited Cells: {(1, 0), (2, 1)}
Current neighbor: (2, 1) - 2_1 - isnt_pit: True - isnt_wumpus: False is_loc: False
Current neighbor: (3, 0) - 3_0 - isnt_pit: True - isnt_wumpus: True 

Exception: Nowhere left to go